### models

In [2]:
import torch
from torchvision.models import resnet50, densenet121, vit_b_16

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self, backbone:str, pretrained=None, output_dim=2):
        super().__init__()
        self.backbone = self.get_backbone(backbone, pretrained)
        self.head = torch.nn.Linear(1000, output_dim)
        #self.head = torch.nn.Sequential(*[
        #    torch.nn.Linear(1000,64),
        #    torch.nn.ReLU(),
        #    torch.nn.Dropout(0.2),
        #    torch.nn.Linear(64,classes),
        #])
        print("parameters=", sum(parameter.numel() for parameter in self.parameters()))

    def get_backbone(self, backbone:str, pretrained:bool):
        if backbone=='resnet50': # 97.8MB, top-1 76.13, (B,3,224,224) -> (B,1000)
            return resnet50(weights='IMAGENET1K_V1' if pretrained else 'default')
        elif backbone=='denseet121': # 30.8MB, top-1 74.43, (B,3,224,224) -> (B,1000)
            return densenet121(weights='IMAGENET1K_V1' if pretrained else 'default')
        elif backbone=='ViT_b_16': # 330.3MB, top-1 81.07, (B,3,224,224) -> (B,1000)
            return vit_b_16(weights='IMAGENET1K_V1' if pretrained else 'default')
        else:
            raise "UnKnown Backbone"
           
    def forward(self, x): # (B,3,224,224) -> (B,output_dim)
        x = self.backbone(x)
        x = self.head(x)
        return x

In [ ]:
#model = MyModel(args.backbone, args.pretrained, args.output_dim)
#if args.resume:
#    model.load_state_dict(torch.load(args.resume))
#else:
#    assert args.mode=='train', f"{args.mode} needs pretrained weights"
#model.to(device)

#torch.save(model.state_dict(), os.path.join(args.results, 'model.pt'))

In [ ]:
D = model.state_dict()
print( [(name,D[name].shape,D[name].numel()) for name in D] )
P = model.named_parameters()
print( [(name,weight.shape,weight.numel()) for name,weight in P] )

with torch.no_grad():
    testInput = torch.rand(2,13).to('cuda')
    start = time.time()
    pred = model(testInput)
    print( round(time.time()-start,5), pred )

In [ ]:
print( list(model.named_parameters())[-1][1] ) # last layer # weight # every time will different
torch.save({'model_state_dict':model.state_dict(),'epoch':0,'loss':99}, 'init.pth')

In [ ]:
model = MyModel().to('cuda')
print( list(model.named_parameters())[-1][1] )

checkpoint = torch.load('./init.pth')
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

print( list(model.named_parameters())[-1][1], epoch, loss )

In [ ]:
"""
model.eval()            # Turn off some layers such as dropout, batch normalization for model inference
with torch.no_grad():   # Turn off gradient calculation
    pass
"""